In [66]:
from polygon import RESTClient
from datetime import datetime, date, timedelta
from tqdm import tqdm
import pandas as pd
import yfinance as yf
import sqlite3

connection = sqlite3.connect('data/bio/candles.db')

In [26]:
df = pd.read_csv('data/bio/finviz_exported.csv').drop(columns=['No.', 'P/E', 'Change', 'Volume'])
df

,Ticker,Company,Sector,Industry,Country,Market Cap,Price
0,LLY,Lilly(Eli) & Co,Healthcare,Drug Manufacturers - General,USA,822096.00,865.00
1,NVO,Novo Nordisk ADR,Healthcare,Biotechnology,Denmark,479498.72,143.63
2,UNH,Unitedhealth Group Inc,Healthcare,Healthcare Plans,USA,455588.10,495.00
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,USA,354094.84,147.13
4,MRK,Merck & Co Inc,Healthcare,Drug Manufacturers - General,USA,333039.20,131.49
...,...,...,...,...,...,...,...
1197,SYRA,Syra Health Corp,Healthcare,Medical Care Facilities,USA,NaN,1.52
1198,TLX,Telix Pharmaceuticals Limited,Healthcare,Biotechnology,Australia,NaN,NaN
1199,TEM,"Tempus AI, Inc.",Healthcare,Diagnostics & Research,USA,NaN,NaN
1200,NTHI,NeOnc Technologies Holdings Inc.,Healthcare,Biotechnology,USA,NaN,NaN


In [27]:
df = df[
    (df['Market Cap'] > 20) &
    (df['Country'] == 'USA') &
    df['Industry'].isin([
        'Biotechnology',
        'Drug Manufacturers - General',
        'Diagnostics & Research',
        'Drug Manufacturers - Specialty & Generic'
    ])
]
df

,Ticker,Company,Sector,Industry,Country,Market Cap,Price
0,LLY,Lilly(Eli) & Co,Healthcare,Drug Manufacturers - General,USA,822096.00,865.00
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,USA,354094.84,147.13
4,MRK,Merck & Co Inc,Healthcare,Drug Manufacturers - General,USA,333039.20,131.49
5,ABBV,Abbvie Inc,Healthcare,Drug Manufacturers - General,USA,301010.21,170.46
7,TMO,Thermo Fisher Scientific Inc.,Healthcare,Diagnostics & Research,USA,221962.54,581.48
...,...,...,...,...,...,...,...
948,INDP,Indaptus Therapeutics Inc,Healthcare,Biotechnology,USA,20.84,2.44
949,IBIO,iBio Inc,Healthcare,Biotechnology,USA,20.69,2.40
952,BNTC,Benitec Biopharma Inc,Healthcare,Biotechnology,USA,20.16,7.41
953,CPIX,Cumberland Pharmaceuticals Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,20.15,1.42


In [28]:
# df.to_csv('data/bio/tickers.csv', index=False)

In [29]:
client = RESTClient(api_key='AKADeecFYwcLpgjEt6rGFpQHbyHi7lBA')

In [60]:
news_df = pd.DataFrame(columns=['ticker', 'published_utc', 'title', 'description', 'related_tickers'])
start_dt = (datetime.now() - timedelta(days=59)).replace(hour=0, minute=0, second=0, microsecond=0)
for ticker in tqdm(df['Ticker'].unique()):
    for news in client.list_ticker_news(ticker, published_utc_gte=start_dt.isoformat()):
        if not news.description or not news.title:
            continue
        if 'FDA' in news.description or 'FDA' in news.title:
            news_df = pd.concat([
                news_df,
                pd.DataFrame([[ticker, news.published_utc, news.title, news.description.replace('\n', ' '), ','.join(news.tickers)]], columns=news_df.columns)
            ], ignore_index=True)

news_df

100%|██████████| 537/537 [02:30<00:00,  3.57it/s]


,ticker,published_utc,title,description,related_tickers
0,LLY,2024-06-11T13:29:00Z,Eli Lilly (LLY) Alzheimer's Drug Donanemab Get...,"Eli Lilly (LLY) Alzheimer's disease drug, dona...","BIIB,LLY,PRTA"
1,LLY,2024-06-10T13:54:00Z,Biogen (BIIB) & Eisai's Leqembi sBLA Gets FDA ...,FDA accepts Biogen's (BIIB) partner Eisai's sB...,"BIIB,LLY,PRTA"
2,LLY,2024-05-15T16:12:00Z,Biogen (BIIB) Partner Begins Filing for Leqemb...,Biogen (BIIB) and Eisai start the rolling subm...,"BIIB,LLY,THRD"
3,LLY,2024-05-10T15:18:00Z,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,"PFE,MRK,LLY"
4,JNJ,2024-05-31T13:33:00Z,"Pharma Stock Roundup: JNJ, MRK M&A Deals, SNY,...",J&J (JNJ) and Merck (MRK) announce the acquisi...,"SNY,AZN,RHHBY,JNJ,MRK"
...,...,...,...,...,...
162,FBIO,2024-04-25T12:30:00Z,Journey Medical Corporation to Present at the ...,"SCOTTSDALE, Ariz., April 25, 2024 (GLOBE NEW...","DERM,FBIO"
163,OCX,2024-04-15T18:11:35Z,Why Ultragenyx Pharmaceutical Shares Are Tradi...,Shares of Ultragenyx Pharmaceutical Inc. (NA...,"YSG,DXYZ,ONVO,KUKE,QTTB,IMCC,LGVN,NVX,SPCB,CMA..."
164,SABS,2024-05-21T11:15:00Z,FDA Provides Clearance to IND Application for ...,FDA Provides Clearance to IND Application for ...,"SABS,SABSW"
165,NERV,2024-06-10T13:42:00Z,AstraZeneca's (AZN) Tagrisso sNDA Gets FDA's P...,The FDA grants priority review to AstraZeneca'...,"AZN,NERV,MRNS,KRYS"


In [61]:
news_df.to_csv('data/bio/fda_news.csv', index=False)

In [64]:
unique_tickers = set(news_df.ticker.unique())
unique_tickers.update( set(pd.read_csv('data/bio/phase_news.csv').ticker.unique()) )
len(unique_tickers)

197

In [ ]:
for ticker in unique_tickers:
    cdf = yf.download(ticker, start=start_dt, end=datetime.now(), interval='2m', prepost=True)
    cdf.drop(columns=['Adj Close']).to_sql(ticker, connection, if_exists='replace')